# Analyse Hitzetage und Tropennächte

Dieses Notebook kann lokal oder **direkt im Browser** auf [![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/meteotest/urban-heat-API-docs/add-heatdays-and-tropicalnights-analysis?labpath=python_data_analysis.ipynb) oder [![Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/meteotest/urban-heat-API-docs/blob/add-heatdays-and-tropicalnights-analysis/python_data_analysis.ipynb) ausgeführt werden.

## Inputs:

In [6]:
# Daten als String in folgendem Format: "YYYY-MM-DDThh:mm:ssZ"
time_from = "2024-05-31T22:00:00Z"
time_to = "2024-06-30T22:00:00Z"

# wenn alle Stationen berücksichtigt werden sollen, dann einfach die Liste leer lassen
# hier ist eine Liste aller Stationen mit Name und ID: https://smart-urban-heat-map.ch/api/v2/stations
station_ids = []

In [7]:
import requests 
import pandas as pd
import geopandas as gpd 
import folium
import branca

In [8]:
def get_stations() -> gpd.GeoDataFrame:
    response = requests.get(
            url=f"https://smart-urban-heat-map.ch/api/v2/stations"
    )
    stations = gpd.read_file(response.text)
    stations.drop(["latestMeasurementDate"], axis=1, inplace=True)

    return stations

def get_station_analysis(time_from: str, time_to: str, station_ids: [str]) -> pd.DataFrame:
    stations = get_stations()
    if station_ids:
        stations = stations[stations["stationId"].isin(station_ids)]
    
    time_from_dt = pd.to_datetime(time_from)
    time_to_dt = pd.to_datetime(time_to)
    time_difference = time_to_dt - time_from_dt
    min_expected_values = round(time_difference.days * 24 * 6 * 0.83)
        
    stations['hitzetage'] = None
    stations['tropennaechte'] = None
    
    for idx, station in stations.iterrows():
        station_id = station.stationId
        response = requests.get(url=f"https://smart-urban-heat-map.ch/api/v2/timeseries?stationId={station_id}&timeFrom={time_from}&timeTo={time_to}")
        
        payload = response.json()
        if payload["values"] is None or not len(payload["values"]): 
            stations = stations.drop(idx)
            continue
            
        if len(payload["values"]) < min_expected_values:
            stations = stations.drop(idx)
            continue
                        
        df = pd.DataFrame(payload["values"])
    
        df["dateObserved"] = pd.to_datetime(df["dateObserved"])
        df["dateObserved"] = df["dateObserved"].dt.tz_convert("Europe/Zurich")
        
        # hier werden die Hitzetage und Tropennächte berechnet
        hitzetage = days_hotter_than(df, 30)
        tropennaechte = nights_hotter_than(df, 20)
        
        stations.at[idx, 'hitzetage'] = hitzetage
        stations.at[idx, 'tropennaechte'] = tropennaechte

    return stations
    
def days_hotter_than(df: pd.DataFrame, temperature: float) -> int:
    daily_maximums = df.groupby(
        df["dateObserved"].dt.date
    ).max()["temperature"]

    return (daily_maximums >= temperature).sum()

def nights_hotter_than(df: pd.DataFrame, temperature: float) -> int:
    df.loc[df['dateObserved'].dt.hour >= 18, 'nightDate'] = df["dateObserved"].dt.date + pd.Timedelta(days=1)
    df.loc[df['dateObserved'].dt.hour < 6, 'nightDate'] = df["dateObserved"].dt.date
        
    nightly_minimums = (
        df.groupby(df['nightDate'])['temperature']
       .min()
    )
        
    return (nightly_minimums > temperature).sum()

In [9]:
stations = get_stations()
stations

,name,stationId,geometry
0,"Ausserholligen 2, ewb",11001,POINT (7.40642 46.94542)
1,Bundesplatz,11002,POINT (7.44353 46.94692)
2,"Breitenrain, Waffenweg",11003,POINT (7.45192 46.96173)
3,Schosshaldenfriedhof 2,11004,POINT (7.47186 46.95339)
4,Monbijou-Park,11005,POINT (7.43462 46.94187)
...,...,...,...
135,Spielplatz Stiglimatt,12007,POINT (7.29920 47.07155)
136,Sportanlage Grien,12008,POINT (7.29574 47.07509)
137,Bahnhofplatz Busswil,12009,POINT (7.31933 47.09871)
138,Reitplatz Grünau,12010,POINT (7.30167 47.07715)


In [10]:
station_analysis = get_station_analysis(time_from, time_to, station_ids)
station_analysis

,name,stationId,geometry,hitzetage,tropennaechte
0,"Ausserholligen 2, ewb",11001,POINT (7.40642 46.94542),3,0
1,Bundesplatz,11002,POINT (7.44353 46.94692),4,1
2,"Breitenrain, Waffenweg",11003,POINT (7.45192 46.96173),4,0
3,Schosshaldenfriedhof 2,11004,POINT (7.47186 46.95339),2,0
4,Monbijou-Park,11005,POINT (7.43462 46.94187),3,0
...,...,...,...,...,...
122,Umland Belp,11125,POINT (7.51184 46.89942),2,0
124,Ittigen Worblentalstrasse,11127,POINT (7.48967 46.97196),4,0
127,Bremgarten Ecke Lindenstrasse / Ritterstrasse ...,11130,POINT (7.43882 46.97791),4,0
133,Marktplatz Lyss,12005,POINT (7.30584 47.07350),4,1


In [11]:
# Hitzetage Map
m = folium.Map(location=[station_analysis.geometry.y.mean(), station_analysis.geometry.x.mean()], zoom_start=13, tiles="CartoDB positron")

colormap = branca.colormap.linear.YlOrRd_09
max_hitzetage = station_analysis.hitzetage.max()
colormap = colormap.scale(0, max_hitzetage).to_step(max_hitzetage) 
colormap.caption = "Anzahl Hitzetage"
colormap.add_to(m)

for idx, station in station_analysis.iterrows():
    folium.CircleMarker(
        location=(station.geometry.y, station.geometry.x),
        radius=5,
        color="black",
        weight=0.5,
        fill=True,
        fill_color=colormap(station.hitzetage),
        fill_opacity=1,
        popup=f"{station.hitzetage:.0f} Hitzetage",
        tooltip=f"{station['name']}: {station.hitzetage:.0f} Hitzetage",
    ).add_to(m)

m

In [12]:
# Tropennächte Map
m = folium.Map(location=[station_analysis.geometry.y.mean(), station_analysis.geometry.x.mean()], zoom_start=13, tiles="CartoDB positron")

colormap = branca.colormap.linear.YlOrRd_09
max_tropennaechte = station_analysis.tropennaechte.max() if station_analysis.tropennaechte.max() > 3 else 3
colormap = colormap.scale(0, max_tropennaechte).to_step(max_tropennaechte) 
colormap.caption = "Anzahl Tropennächte"
colormap.add_to(m)

for idx, station in station_analysis.iterrows():
    folium.CircleMarker(
        location=(station.geometry.y, station.geometry.x),
        radius=5,
        color="black",
        weight=0.5,
        fill=True,
        fill_color=colormap(station.tropennaechte),
        fill_opacity=1,
        tooltip=f"{station['name']}: {station.tropennaechte:.0f} Tropennächte",
    ).add_to(m)

m